<a href="https://colab.research.google.com/github/shaodwing/nmist-fashion/blob/master/Fashion_NMIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from tqdm import tqdm
from scipy import misc
import tensorflow as tf
from keras.datasets import fashion_mnist

#MobileNet by keras
from keras.applications.mobilenet import MobileNet
from keras.layers import Input,Dense,Dropout,Lambda
from keras.models import Model
from keras import backend as K


#MobileNetV2 by leonard
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

#Tiny  darknet
from keras import utils
from keras.callbacks import Callback
from keras.layers import BatchNormalization, Conv2D, GlobalAveragePooling2D, Activation, LeakyReLU, MaxPooling2D
from keras.optimizers import SGD


#Load Data

In [0]:
np.random.seed(2019)
tf.set_random_seed(2019)

(X_train, Y_train ), (X_test, Y_test ) = fashion_mnist.load_data()

#MobileNet de keras du mec qui a participé au fashion mnist


In [11]:
"""
#Credit
#Code from 苏剑林, on https://kexue.fm/archives/4556
height,width = 56,56


input_image = Input(shape=(height,width))
input_image_ = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,3),3,3))(input_image)
base_model = MobileNet( input_shape=None,
                        alpha=1.0,
                        depth_multiplier=0.5,#1,
                        dropout=1e-3,
                        include_top=True,
                        weights= None,#'imagenet',
                        input_tensor=input_image_,
                        pooling='avg',
                        classes=1000)
output = Dropout(0.5)(base_model.output)
predict = Dense(10, activation='softmax')(output)

model = Model(inputs=input_image, outputs=predict)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

X_train = X_train.reshape((-1,28,28))
X_train = np.array([( np.resize(x, (height,width))).astype(float) for x in tqdm(iter(X_train))])/255.

X_test = X_test.reshape((-1,28,28))
X_test = np.array([( np.resize(x, (height,width))).astype(float) for x in tqdm(iter(X_test))])/255.

model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))
"""

"\n#Credit\n#Code from 苏剑林, on https://kexue.fm/archives/4556\nheight,width = 56,56\n\n\ninput_image = Input(shape=(height,width))\ninput_image_ = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,3),3,3))(input_image)\nbase_model = MobileNet( input_shape=None,\n                        alpha=1.0,\n                        depth_multiplier=0.5,#1,\n                        dropout=1e-3,\n                        include_top=True,\n                        weights= None,#'imagenet',\n                        input_tensor=input_image_,\n                        pooling='avg',\n                        classes=1000)\noutput = Dropout(0.5)(base_model.output)\npredict = Dense(10, activation='softmax')(output)\n\nmodel = Model(inputs=input_image, outputs=predict)\nmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])\nmodel.summary()\n\nX_train = X_train.reshape((-1,28,28))\nX_train = np.array([( np.resize(x, (height,width))).astype(float) for x in tqdm(iter(

#MobileNet de Leo

In [12]:
"""
batch_size = 128
num_classes = 10 
epochs = 12


x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255


model = Sequential()



model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

score=model.evaluate(x_test, y_test,verbose=0)
print('test loss:', score[0])
print('Test accuracy:', score[1])

model.save('Test_MNIST_MLP.h5')
"""

"\nbatch_size = 128\nnum_classes = 10 \nepochs = 12\n\n\nx_train = x_train.astype('float32')\nx_test  = x_test.astype('float32')\nx_train /= 255\nx_test  /= 255\n\n\nmodel = Sequential()\n\n\n\nmodel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))\n\nscore=model.evaluate(x_test, y_test,verbose=0)\nprint('test loss:', score[0])\nprint('Test accuracy:', score[1])\n\nmodel.save('Test_MNIST_MLP.h5')\n"

#Tiny dark net Leo

In [13]:
#Used Joseph Redmon on tiny darknet to produce my work

batch_size = 128
num_classes = 10 
epochs = 40
img_cols = X_train.shape[1]
img_rows = X_train.shape[2]


if K.image_data_format() == 'channels_first':
  X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols);
  X_test  = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols);
  input_shape = (1, img_rows, img_cols);
else: 
  X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1);
  X_test  = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1);
  input_shape = (img_rows, img_cols, 1);


X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_train /= 255
X_test  /= 255

Y_train = utils.to_categorical(Y_train, num_classes)
Y_test  = utils.to_categorical(Y_test, num_classes)

print(X_train.shape)
print(X_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [0]:
#Function
def layerConv(output, filters, size, stride, pad, batch_normalization=True, activation='LeakyReLU'):
  output = Conv2D(kernel_size = (size,size), filters = filters, strides=stride, padding=pad)(output)
  if batch_normalization:
    output = BatchNormalization()(output);
  
  #Activation layer
  if activation=='LeakyReLU':
    output = LeakyReLU(alpha = 0.1)(output)
  else:
    output = Activation(activation)(output)
  
  return output


In [18]:
#LAYERS
#First set of convolution (9-12)
input_img = Input(shape=input_shape)
output = layerConv(output=input_img, filters=128, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=32 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=32 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output   , filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')

#Max pooling to get 14x14 feature (13)
output = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(output)

#Second set of convolution (14-19)
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output, filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #512
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU')
output = layerConv(output=output, filters=256, size=3, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #512
output = layerConv(output=output, filters=64 , size=1, batch_normalization=True, stride=1, pad="same", activation='LeakyReLU') #128

#Last layer to get an output of 10 class (19+)
output = layerConv(output=output, filters=10, size=1, batch_normalization=False, stride=1, pad="same", activation='linear')
output = GlobalAveragePooling2D()(output)
output = Activation('softmax')(output)
#output = cost layer

model = Model(input_img, output)
print(model.summary())



opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2/epochs)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size=256,
          shuffle=True, 
          validation_data=(X_test, Y_test))

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 28, 28, 128)       1280      
_________________________________________________________________
batch_normalization_47 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
leaky_re_lu_46 (LeakyReLU)   (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 28, 28, 32)        4128      
_________________________________________________________________
batch_normalization_48 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 28, 28, 32)        0   